# Python HANA ML APL

<div class="alert alert-block alert-info">
<b>Predicting Car Insurance Fraud.</b> <br>
</div>

### Create an HANA Dataframe containing historical Insurance Claims

In [ ]:
from hana_ml import dataframe as hd
conn = hd.ConnectionContext(userkey='MLMDA_KEY')
hdf_new = conn.table('AUTO_CLAIMS_NEW', schema='APL_SAMPLES')

In [ ]:
hdf_new.head(3).collect()

### Load Trained Model

In [ ]:
from hana_ml.model_storage import ModelStorage
model_storage = ModelStorage(connection_context=conn, schema='USER_APL')
apl_model = model_storage.load_model(name='My Fraud Model')
apl_model.get_model_info()

In [ ]:
df = apl_model.get_summary().filter("KEY in ('ModelVariableCount', 'ModelSelectedVariableCount', 'ModelRecordCount', 'ModelBuildDate')").collect()
df['KEY'] = df['KEY'].str.replace('Model', '').str.replace('Selected', 'Selected ')
df['KEY'] = df['KEY'].str.replace('Count', ' Count').str.replace('Date', ' Date')
df = df[['KEY','VALUE']]
df.columns = ['Property', 'Value']
df.style.hide(axis='index')

### Make Predictions on New Claims

In [ ]:
apl_model.set_params(extra_applyout_settings=
{ 'APL/ApplyExtraMode': 'Advanced Apply Settings', 
  'APL/ApplyPredictedValue': 'true', 
  'APL/ApplyProbability': 'true', 
  'APL/ApplyDecision': 'true', 
  'APL/ApplyReasonCode/TopCount': '3', 
  'APL/ApplyReasonCode/ShowStrengthValue': 'false', 
  'APL/ApplyReasonCode/ShowStrengthIndicator': 'false' }
)
df = apl_model.predict(hdf_new).collect()

In [ ]:
col_dict = {'PREDICTED': 'Target Predicted', 
        'gb_score_IS_FRAUD': 'Score', 
        'gb_proba_IS_FRAUD': 'Probability'}
df.rename(columns=col_dict, inplace=True)
df.columns = [hdr.replace("gb_", "") for hdr in df.columns]
format_dict = {'Probability': '{:,.1%}','Score':'{0:,.2f}'}
df.head(7).style.format(format_dict).hide(axis='index')